In [1]:
from src.train import SphereClassifier, WhaleDataModule
from src.dataset import load_df
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from config.config import Config, load_config

cuda = torch.device("cuda:0")

## Load model

In [2]:
# model = SphereClassifier.load_from_checkpoint(
#     checkpoint_path="/app/sandbox/happy_whale/kaggle-happywhale-1st-place/result/b6_new/1/last.ckpt"
# )
# model.to(cuda)
# model.eval()


# image = torch.rand(1, 3, 528, 528).to(cuda)
# logits_ids, logits_species = model(image)

## Compute embs on train ds

In [3]:
cfg = load_config("config/efficientnet_b6_new.yaml", "config/default.yaml")

used default config lr_backbone: 0.0016
used default config lr_head: 0.016
used default config lr_decay_scale: 0.01
used default config num_classes: 15587
used default config num_species_classes: 26
used default config pretrained: True
used default config val_bbox: fullbody
used default config test_bboxes: ['fullbody', 'fullbody_charm']
used default config bboxes: {'fullbody_charm': 0.15, 'fullbody': 0.6, 'backfin': 0.15, 'detic': 0.05, 'none': 0.05}
used default config bbox_conf_threshold: 0.01
used default config n_data: -1
used default config global_pool: {'arch': 'GeM', 'p': 3, 'train': False}
used default config normalization: batchnorm
used default config optimizer: AdamW
used default config loss_fn: CrossEntropy
used default config loss_id_ratio: 0.437338
used default config margin_coef_id: 0.27126
used default config margin_coef_species: 0.226253
used default config margin_power_id: -0.364399
used default config margin_power_species: -0.720133
used default config s_id: 20.9588


In [4]:
df = load_df("input", cfg, "train.csv", True)
data_module = WhaleDataModule(
    df,
    cfg,
    f"input/train_images",
    cfg.val_bbox,
    -1,
)

detic low conf: 0 / 51033
fullbody low conf: 0 / 51033
fullbody_charm low conf: 10 / 51033
backfin low conf: 1587 / 51033


In [5]:
train_dataset = data_module.get_dataset(df, False)
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=32,
    pin_memory=True,
    drop_last=False,
)

In [6]:
# predictions = []
# model.eval()

# for batch in tqdm(train_loader):
#     images = batch['image'].to(cuda)
#     feats = F.normalize(model.get_feat(images), p=2.0, dim=1)
#     predictions.append(feats.detach().cpu())
# embs = torch.cat(predictions, axis=0).numpy()
# np.savez(f"whale_train_emb.npz", embs=embs)

## Create Whale Train OSFR protocol

In [41]:
train_dataset.ids  # defines unique id of image. id \in [0, 15587)
a = np.load("whale_train_emb.npz")

In [42]:
a["embs"].shape

(51033, 776)

In [39]:
unique_ids, count_ids = np.unique(train_dataset.ids, return_counts=True)

out_of_gallery_ids = unique_ids[np.argsort(count_ids)[:9258]]  # single image ids
in_gallery_ids = unique_ids[np.argsort(count_ids)[9258:]]
assert np.sum(train_dataset.ids == 5597) == 1

In [ ]:
image_path_to_template_id = {}
image_path_to_subject_id = {}

# for image_name, image_id in zip(train_dataset.x_paths, train_dataset.ids):

In [10]:
from pathlib import Path

ds_dir = Path("/app/datasets/whale_train")
ds_dir.mkdir(exist_ok=True)
meta_path = ds_dir / "meta"
meta_path.mkdir(exist_ok=True)